In [ ]:
from the_well.data import WellDataset

the_well = WellDataset(
    well_base_path="./the_well/datasets",
    well_dataset_name="active_matter",
    well_split_name="test",
    n_steps_input=1,
    n_steps_output=1,
    
)

In [ ]:
from torch.utils.data import DataLoader

train_loader = DataLoader(the_well)

batch = next(iter(train_loader))


In [ ]:
batch["input_fields"].shape, batch["output_fields"].shape

In [ ]:

batch["input_fields"][0, 0, :, :, 0].shape

In [ ]:

import matplotlib.pyplot as plt

plt.imshow(batch["input_fields"][0, 0, :, :, 0])


In [ ]:
from neuralop.models import FNO, FNO2d


# Q: what happens if there is more than one channel? How are these combined
model = FNO2d(
    n_modes_height=16,
    n_modes_width=16,
    hidden_channels=16,
    in_channels=1,  # Single scalar field input
    out_channels=1,  # Single scalar field output
    
)

In [ ]:
x = batch["input_fields"][:1, 0, :, :, :1]  # [batch, height, width, channels]
x = x.permute(0, 3, 1, 2)  # Convert to [batch, channels, height, width]
x.shape

In [ ]:
model(x).shape

In [ ]:
from torch.optim import AdamW
from torch.nn import MSELoss as RMSELoss
optimizer = AdamW(
    model.parameters(),
    lr=8e-3,
    weight_decay=1e-4
)
loss_fn = RMSELoss()

for i, batch in enumerate(iter(the_well)):
    x = batch["input_fields"][:1, :, :, :1]  # [batch, height, width, channels]
    y = batch["output_fields"][:1, :, :, :1]  # [batch, height, width, channels]
    # transpose to correct channel location
    x = x.permute(0, 3, 1, 2)  # Convert to [batch, channels, height, width]
    y = y.permute(0, 3, 1, 2)  #

    # Get prediction
    y_pred = model(x)

    # Get loss
    loss = loss_fn(y_pred, y)
    
    print(f"Batch {i+1}, Loss: {loss.item():.6f}")

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()